# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 12 2023 12:21PM,264578,19,ADV80020814-Hold_Rx(IA),"AdvaGen Pharma, Ltd",Released
1,Jun 12 2023 12:21PM,264578,19,ADV80020817,"AdvaGen Pharma, Ltd",Released
2,Jun 12 2023 12:14PM,264577,10,9564613,"Snap Medical Industries, LLC",Released
3,Jun 12 2023 12:14PM,264577,10,9564614,"Snap Medical Industries, LLC",Released
4,Jun 12 2023 12:14PM,264577,10,9565144,"Snap Medical Industries, LLC",Released
5,Jun 12 2023 12:14PM,264577,10,9565146,"Snap Medical Industries, LLC",Released
6,Jun 12 2023 12:14PM,264577,10,9565147,"Snap Medical Industries, LLC",Released
7,Jun 12 2023 12:14PM,264577,10,9565149,"Snap Medical Industries, LLC",Released
8,Jun 12 2023 12:14PM,264577,10,9565150,"Snap Medical Industries, LLC",Released
9,Jun 12 2023 12:14PM,264577,10,9565151,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,264574,Released,14
20,264575,Released,1
21,264576,Released,1
22,264577,Released,9
23,264578,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264574,9.0,14.0
264575,NaN,1.0
264576,NaN,1.0
264577,NaN,9.0
264578,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264535,0.0,1.0
264537,0.0,1.0
264543,0.0,2.0
264544,0.0,16.0
264547,18.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264535,0,1
264537,0,1
264543,0,2
264544,0,16
264547,18,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264535,0,1
1,264537,0,1
2,264543,0,2
3,264544,0,16
4,264547,18,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264535,,1
1,264537,,1
2,264543,,2
3,264544,,16
4,264547,18,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd"
2,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC"
11,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC"
12,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc."
13,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC"
36,Jun 12 2023 11:51AM,264572,10,"Senseonics, Incorporated"
37,Jun 12 2023 11:49AM,264571,10,"Methapharm, Inc."
45,Jun 12 2023 11:48AM,264570,10,"Methapharm, Inc."
46,Jun 12 2023 11:44AM,264568,19,"GCH Granules USA, Inc."
47,Jun 12 2023 11:42AM,264567,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd",,2
1,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC",,9
2,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC",,1
3,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc.",,1
4,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC",9,14
5,Jun 12 2023 11:51AM,264572,10,"Senseonics, Incorporated",,1
6,Jun 12 2023 11:49AM,264571,10,"Methapharm, Inc.",,8
7,Jun 12 2023 11:48AM,264570,10,"Methapharm, Inc.",,1
8,Jun 12 2023 11:44AM,264568,19,"GCH Granules USA, Inc.",,1
9,Jun 12 2023 11:42AM,264567,10,"CLINUVEL, Inc.",,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd",2,
1,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC",9,
2,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC",1,
3,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc.",1,
4,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC",14,9
5,Jun 12 2023 11:51AM,264572,10,"Senseonics, Incorporated",1,
6,Jun 12 2023 11:49AM,264571,10,"Methapharm, Inc.",8,
7,Jun 12 2023 11:48AM,264570,10,"Methapharm, Inc.",1,
8,Jun 12 2023 11:44AM,264568,19,"GCH Granules USA, Inc.",1,
9,Jun 12 2023 11:42AM,264567,10,"CLINUVEL, Inc.",4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd",2,
1,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC",9,
2,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC",1,
3,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc.",1,
4,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC",14,9


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd",2.0,NaN
1,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC",9.0,NaN
2,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC",1.0,NaN
3,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc.",1.0,NaN
4,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC",14.0,9.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 12:21PM,264578,19,"AdvaGen Pharma, Ltd",2.0,0.0
1,Jun 12 2023 12:14PM,264577,10,"Snap Medical Industries, LLC",9.0,0.0
2,Jun 12 2023 12:08PM,264576,10,"Snap Medical Industries, LLC",1.0,0.0
3,Jun 12 2023 12:00PM,264575,10,"Citieffe, Inc.",1.0,0.0
4,Jun 12 2023 12:00PM,264574,19,"Innogenix, LLC",14.0,9.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2910193,52.0,0.0
15,1058227,56.0,20.0
18,264535,1.0,0.0
19,1058275,18.0,9.0
21,264549,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2910193,52.0,0.0
1,15,1058227,56.0,20.0
2,18,264535,1.0,0.0
3,19,1058275,18.0,9.0
4,21,264549,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,52.0,0.0
1,15,56.0,20.0
2,18,1.0,0.0
3,19,18.0,9.0
4,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,15,Released,56.0
2,18,Released,1.0
3,19,Released,18.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21
Status,,,,,
Executing,0.0,20.0,0.0,9.0,1.0
Released,52.0,56.0,1.0,18.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21
0,Executing,0.0,20.0,0.0,9.0,1.0
1,Released,52.0,56.0,1.0,18.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21
0,Executing,0.0,20.0,0.0,9.0,1.0
1,Released,52.0,56.0,1.0,18.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()